In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

## Read dataset

In [80]:
sales_receipt_df = pd.read_csv('./datasets/201904 sales reciepts.csv')
products_df = pd.read_csv('./datasets/product.csv')

In [81]:
sales_receipt_df.head(2)

,transaction_id,transaction_date,transaction_time,sales_outlet_id,staff_id,customer_id,instore_yn,order,line_item_id,product_id,quantity,line_item_amount,unit_price,promo_item_yn
0,7,2019-04-01,12:04:43,3,12,558,N,1,1,52,1,2.5,2.5,N
1,11,2019-04-01,15:54:39,3,17,781,N,1,1,27,2,7.0,3.5,N


In [82]:
products_df.head(2)

,product_id,product_group,product_category,product_type,product,product_description,unit_of_measure,current_wholesale_price,current_retail_price,tax_exempt_yn,promo_yn,new_product_yn
0,1,Whole Bean/Teas,Coffee beans,Organic Beans,Brazilian - Organic,It's like Carnival in a cup. Clean and smooth.,12 oz,14.4,$18.00,Y,N,N
1,2,Whole Bean/Teas,Coffee beans,House blend Beans,Our Old Time Diner Blend,Out packed blend of beans that is reminiscent ...,12 oz,14.4,$18.00,Y,N,N


## Data Wrangling

In [83]:
# filter out the relevent data
sales_receipt_df = sales_receipt_df[['transaction_id','transaction_date','sales_outlet_id','customer_id','product_id','quantity']]
products_df = products_df[['product_id','product_category','product']]

In [84]:
# merging the datasets
df = pd.merge(sales_receipt_df, products_df, how='left', on='product_id')
df.head(2)

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product
0,7,2019-04-01,3,558,52,1,Tea,Traditional Blend Chai Rg
1,11,2019-04-01,3,781,27,2,Coffee,Brazilian Lg


### removing the sizes Lg, Sm, Rg and irrelevent items from the dataset and finding the category

In [85]:
# this is the before sizes
print("before :: ",df[df['product'].str.contains('Brazilian')]['product'].unique())

# replacing the sizes value with nothing
df['product'] = df['product'].str.replace('Lg', '')
df['product'] = df['product'].str.replace('Sm', '')
df['product'] = df['product'].str.replace('Rg', '')

# this is after
print("after ::", df[df['product'].str.contains('Brazilian')]['product'].unique())

before ::  ['Brazilian Lg' 'Brazilian Sm' 'Brazilian Rg' 'Brazilian - Organic']
after :: ['Brazilian ' 'Brazilian - Organic']


In [86]:
print(sorted(list(df['product'].unique())))

['Almond Croissant', 'Brazilian ', 'Brazilian - Organic', 'Cappuccino', 'Cappuccino ', 'Carmel syrup', 'Chili Mayan', 'Chocolate Chip Biscotti', 'Chocolate Croissant', 'Chocolate syrup', 'Civet Cat', 'Columbian Medium Roast', 'Columbian Medium Roast ', 'Cranberry Scone', 'Croissant', 'Dark chocolate', 'Dark chocolate ', 'Earl Grey', 'Earl Grey ', 'English Breakfast', 'English Breakfast ', 'Espresso Roast', 'Espresso shot', 'Ethiopia', 'Ethiopia ', 'Ginger Biscotti', 'Ginger Scone', 'Guatemalan Sustainably Grown', 'Hazelnut Biscotti', 'Hazelnut syrup', 'I Need My Bean! Diner mug', 'I Need My Bean! Latte cup', 'I Need My Bean! T-shirt', 'Jamacian Coffee River', 'Jamaican Coffee River ', 'Jumbo Savory Scone', 'Latte', 'Latte ', 'Lemon Grass', 'Lemon Grass ', 'Morning Sunrise Chai', 'Morning Sunrise Chai ', 'Oatmeal Scone', 'Organic Decaf Blend', 'Our Old Time Diner Blend', 'Our Old Time Diner Blend ', 'Ouro Brasileiro shot', 'Peppermint', 'Peppermint ', 'Primo Espresso Roast', 'Scottish C

In [87]:
# the item which i select from the dataset
products_to_consider = ['Cappuccino','Latte ','Espresso shot','Dark chocolate ','Sugar Free Vanilla syrup','Chocolate syrup',
                        'Carmel syrup','Hazelnut syrup','Ginger Scone','Chocolate Croissant','Jumbo Savory Scone','Cranberry Scone',
                        'Hazelnut Biscotti','Croissant','Almond Croissant','Oatmeal Scone','Chocolate Chip Biscotti','Ginger Biscotti']

# filtering the dataset
df = df[df['product'].isin(products_to_consider)]
df.head(2)

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product
16,108,2019-04-01,3,65,40,1,Coffee,Cappuccino
17,112,2019-04-01,3,90,37,2,Coffee,Espresso shot


In [88]:
# finding the product category with the corrosponding product
df[['product','product_category']].drop_duplicates().reset_index(drop=True)

,product,product_category
0,Cappuccino,Coffee
1,Espresso shot,Coffee
2,Dark chocolate,Drinking Chocolate
3,Oatmeal Scone,Bakery
4,Jumbo Savory Scone,Bakery
5,Latte,Coffee
6,Chocolate Chip Biscotti,Bakery
7,Ginger Biscotti,Bakery
8,Chocolate Croissant,Bakery
9,Hazelnut Biscotti,Bakery


## Clean Transnation

Now some trasnactions id goes to multiple customer because here is only one product per row, so 
<br>if they are sitting together and pay as a group then the same id can go to multiple customers
<br> or if a person order multiple items
<br> which can confuse recommandation engine so we concatinate transationid and customer id to make a new transation number.

In [89]:
df['transaction'] = df['transaction_id'].astype('str') + '_' + df['customer_id'].astype('str')
df.head(2)

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product,transaction
16,108,2019-04-01,3,65,40,1,Coffee,Cappuccino,108_65
17,112,2019-04-01,3,90,37,2,Coffee,Espresso shot,112_90


### removing the customers which only bought ones and never return back because we don't need them for recommandation engine

In [90]:
num_of_itemes_for_transaction = df['transaction'].value_counts().reset_index()
num_of_itemes_for_transaction.head()

,transaction,count
0,209_0,31
1,206_0,30
2,204_0,27
3,208_0,25
4,203_0,24


In [91]:
# filter tensactions with more than 2 items
valid_transactions = num_of_itemes_for_transaction[num_of_itemes_for_transaction['count'] > 1]['transaction'].to_list()
df = df[df['transaction'].isin(valid_transactions)]
df.head(2)

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product,transaction
54,296,2019-04-01,3,328,39,1,Coffee,Latte,296_328
55,296,2019-04-01,3,328,79,1,Bakery,Jumbo Savory Scone,296_328


In [92]:
df.shape

(8451, 9)